In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 8.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660426 sha256=516a3ec755841a91c3bd8b7413c388b9ccdea8c632720f57bef45305992681a0
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [2]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray, compiler

# Define CUDA kernel for matrix multiplication
kernel_code = """
__global__ void matrix_mult(float *A, float *B, float *C, int size) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        float sum = 0.0f;
        for (int k = 0; k < size; k++) {
            sum += A[row * size + k] * B[k * size + col];
        }
        C[row * size + col] = sum;
    }
}
"""

# Compile the kernel
mod = compiler.SourceModule(kernel_code)
matrix_mult = mod.get_function("matrix_mult")

# Matrix dimensions
N = 5  # Small size for readability

# Initialize matrices (identity matrices)
A = np.eye(N, dtype=np.float32)
B = np.eye(N, dtype=np.float32)
C = np.zeros((N, N), dtype=np.float32)

# Print input matrices
print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Allocate GPU memory
A_gpu = gpuarray.to_gpu(A)
B_gpu = gpuarray.to_gpu(B)
C_gpu = gpuarray.zeros((N, N), dtype=np.float32)

# Define block/grid dimensions
block_size = (16, 16, 1)  # 16x16 threads per block
grid_size = (
    (N + block_size[0] - 1) // block_size[0],
    (N + block_size[1] - 1) // block_size[1],
    1
)

# Launch kernel
matrix_mult(
    A_gpu, B_gpu, C_gpu,
    np.int32(N),
    block=block_size,
    grid=grid_size
)

# Copy result back to CPU
C = C_gpu.get()

# Print result matrix
print("\nMatrix C (Result):")
print(C)

# Verify results
tolerance = 1e-5
correct = np.allclose(C, np.eye(N), atol=tolerance)

if correct:
    print("\nMatrix multiplication succeeded!")
else:
    print("\nMatrix multiplication failed!")
    # Show mismatched indices
    mismatch_indices = np.where(~np.isclose(C, np.eye(N), atol=tolerance))
    for i, j in zip(*mismatch_indices):
        print(f"Mismatch at ({i}, {j}): Expected 0.0, Got {C[i,j]}")

Matrix A:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Matrix B:
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Matrix C (Result):
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Matrix multiplication succeeded!
